In [1]:
%matplotlib inline
import torch
from torch import nn
from torch.nn import functional as F
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import utils
import torchvision
from torchvision.io import image
from torchvision.transforms.functional import to_pil_image
import pandas as pd
import time
import numpy as np
import collections
import re
import random
import math
import json
import multiprocessing

In [2]:
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """获取输入序列的词元及其片段索引"""
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    # 0和1分别标记片段A和B
    segments = [0] * (len(tokens_a) + 2)
    if tokens_b is not None:
        tokens += tokens_b + ['<sep>']
        segments += [1] * (len(tokens_b) + 1)
    return tokens, segments

In [3]:
class BERTEncoder(nn.Module):
    """BERT编码器"""
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768, **kwargs):
        super(BERTEncoder, self).__init__(**kwargs)
        self.token_embedding = nn.Embedding(vocab_size, num_hiddens)
        self.segment_embedding = nn.Embedding(2, num_hiddens)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module(f"{i}", utils.EncoderBlock(key_size, query_size, value_size, num_hiddens, norm_shape,
                                                            ffn_num_input, ffn_num_hiddens, num_heads, dropout, True))
        # 在BERT中，位置嵌入是可学习的，因此我们创建一个足够长的位置嵌入参数
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len, num_hiddens))

    def forward(self, tokens, segments, valid_lens):
        # 在以下代码段中，X的形状保持不变：（批量大小，最大序列长度，num_hiddens）
        X = self.token_embedding(tokens) + self.segment_embedding(segments)
        X = X + self.pos_embedding.data[:, :X.shape[1], :]
        for blk in self.blks:
            X = blk(X, valid_lens)
        return X

In [4]:
vocab_size, num_hiddens, ffn_num_hiddens, num_heads = 10000, 768, 1024, 4
norm_shape, ffn_num_input, num_layers, dropout = [768], 768, 2, 0.2
encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, num_layers, dropout)

In [5]:
# tokens定义为长度为8的2个输入序列，其中每个词元是词表的索引
tokens = torch.randint(low=0, high=vocab_size, size=(2, 8))
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])
encoded_X = encoder(tokens, segments, None)
encoded_X.shape

torch.Size([2, 8, 768])

In [6]:
# 预训练包括两个任务：掩蔽语言模型和下一句预测。前者能够编码双向上下文来表示单词，而后者则显式地建模文本对之间的逻辑关系。

# 掩蔽语言模型（Masked Language Modeling）

In [7]:
# 随机选择15%的词元作为预测的掩蔽词元

#1, 80%时间为特殊的“<mask>“词元（例如，“this movie is great”变为“this movie is<mask>”；

#2, 10%时间为随机词元（例如，“this movie is great”变为“this movie is drink”）；

#3, 10%时间内为不变的标签词元（例如，“this movie is great”变为“this movie is great”）。

In [8]:
class MaskLM(nn.Module):
    """BERT的掩蔽语言模型任务"""
    def __init__(self, vocab_size, num_hiddens, num_inputs=768, **kwargs):
        super(MaskLM, self).__init__(**kwargs)
        self.mlp = nn.Sequential(nn.Linear(num_inputs, num_hiddens),
                                 nn.ReLU(),
                                 nn.LayerNorm(num_hiddens),
                                 nn.Linear(num_hiddens, vocab_size))

    def forward(self, X, pred_positions):
        # X:BERTEncoder的编码结果, pred_positions：用于预测的词元位置
        num_pred_positions = pred_positions.shape[1]
        pred_positions = pred_positions.reshape(-1)
        batch_size = X.shape[0]
        batch_idx = torch.arange(0, batch_size)
        # 假设batch_size=2，num_pred_positions=3
        # 那么batch_idx是np.array（[0,0,0,1,1,1]）
        batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions)
        masked_X = X[batch_idx, pred_positions]
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))
        # masked_X.shape [2, 3, 768]
        mlm_Y_hat = self.mlp(masked_X)
        return mlm_Y_hat

In [9]:
mlm = MaskLM(vocab_size, num_hiddens)
mlm_positions = torch.tensor([[1, 5, 2], [6, 1, 5]])
mlm_Y_hat = mlm(encoded_X, mlm_positions)
mlm_Y_hat.shape

torch.Size([2, 3, 10000])

In [10]:
# 通过掩码下的预测词元的真实标签mlm_Y，我们可以计算在BERT预训练中的遮蔽语言模型任务的交叉熵损失。
mlm_Y = torch.tensor([[7, 8, 9], [10, 20, 30]])
loss = nn.CrossEntropyLoss(reduction='none')
mlm_l = loss(mlm_Y_hat.reshape((-1, vocab_size)), mlm_Y.reshape(-1))
mlm_l.shape

torch.Size([6])

# 下一句预测（Next Sentence Prediction）

In [11]:
# BERT在预训练中考虑了一个二元分类任务————下一句预测。
# 在为预训练生成句子对时，有一半的时间它们确实是标签为“真”的连续句子；在另一半的时间里，第二个句子是从语料库中随机抽取的，标记为“假”。

In [12]:
# 预测第二个句子是否是BERT输入序列中第一个句子的下一个句子
class NextSentencePred(nn.Module):
    """BERT的下一句预测任务"""
    def __init__(self, num_inputs, **kwargs):
        super(NextSentencePred, self).__init__(**kwargs)
        self.output = nn.Linear(num_inputs, 2)

    def forward(self, X):
        # X的形状：(batchsize,num_hiddens)
        return self.output(X)

In [13]:
print(encoded_X.shape)
encoded_X = torch.flatten(encoded_X, start_dim=1)
# NSP的输入形状:(batchsize，num_hiddens)
nsp = NextSentencePred(encoded_X.shape[-1])
nsp_Y_hat = nsp(encoded_X)
print(encoded_X.shape)
print(nsp_Y_hat.shape)

torch.Size([2, 8, 768])
torch.Size([2, 6144])
torch.Size([2, 2])


In [14]:
# 计算两个二元分类的交叉熵损失
nsp_y = torch.tensor([0, 1])
nsp_l = loss(nsp_Y_hat, nsp_y)
nsp_l.shape

torch.Size([2])

# 整合代码

In [15]:
# 在预训练BERT时，最终的损失函数是掩蔽语言模型损失函数和下一句预测损失函数的线性组合。
# 现在我们可以通过实例化三个类BERTEncoder、MaskLM和NextSentencePred来定义BERTModel类。
# 前向推断返回编码后的BERT表示encoded_X、掩蔽语言模型预测mlm_Y_hat和下一句预测nsp_Y_hat。

class BERTModel(nn.Module):
    """BERT模型"""
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768, hid_in_features=768, mlm_in_features=768, nsp_in_features=768):
        super(BERTModel, self).__init__()
        self.encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, num_layers, 
                                   dropout, max_len=max_len, key_size=key_size, query_size=query_size, value_size=value_size)
        self.hidden = nn.Sequential(nn.Linear(hid_in_features, num_hiddens),
                                    nn.Tanh())
        self.mlm = MaskLM(vocab_size, num_hiddens, mlm_in_features)
        self.nsp = NextSentencePred(nsp_in_features)

    def forward(self, tokens, segments, valid_lens=None, pred_positions=None):
        encoded_X = self.encoder(tokens, segments, valid_lens)
        if pred_positions is not None:
            mlm_Y_hat = self.mlm(encoded_X, pred_positions)
        else:
            mlm_Y_hat = None
        # 用于下一句预测的多层感知机分类器的隐藏层，0是“<cls>”标记的索引
        nsp_Y_hat = self.nsp(self.hidden(encoded_X[:, 0, :]))
        return encoded_X, mlm_Y_hat, nsp_Y_hat

# BERT预训练数据集处理

In [16]:
# WikiText-2数据集（1）保留了原来的标点符号，适合于下一句预测；（2）保留了原来的大小写和数字；

# 在WikiText-2数据集中，每行代表一个段落，其中在任意标点符号及其前面的词元之间插入空格。
# 保留至少有两句话的段落。为了简单起见，我们仅使用句号作为分隔符来拆分句子。

In [17]:
data_dir_path = Path(r'../data/wikitext-2/')

In [18]:
def _read_wiki(data_dir):
    file_path = data_dir / 'wiki.train.tokens'
    with open(file_path.as_posix(), 'r', encoding='utf8') as f:
        lines = f.readlines()
    # 大写字母转换为小写字母
    paragraphs = [line.strip().lower().split(' . ') for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs

In [19]:
# 生成下一句预测任务的数据
def _get_next_sentence(sentence, next_sentence, paragraphs):
    if random.random() < 0.5:
        is_next = True
    else:
        # paragraphs是三重列表的嵌套
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next

In [20]:
# 生成用于下一句预测的训练样本
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(paragraph[i], paragraph[i + 1], paragraphs)
        # 考虑1个'<cls>'词元和2个'<sep>'词元
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        tokens, segments = get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

In [21]:
# 生成遮蔽语言模型任务的数据——替换mlm中需要mask的词元
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds, vocab):
    # candidate_pred_positions：候选预测位置，不包括特殊词元的BERT输入序列的词元索引的列表（特殊词元在遮蔽语言模型任务中不被预测）
    # num_mlm_preds：指示预测的数量（选择15%要预测的随机词元）
    
    # 为遮蔽语言模型的输入创建新的词元副本，后续mlm_input_tokens可能包含替换的“<mask>”或随机词元
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    # 打乱后用于在遮蔽语言模型任务中获取15%的随机词元进行预测
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        # 80%的时间：将词替换为“<mask>”词元
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            # 10%的时间：保持词不变
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            # 10%的时间：用随机词替换该词
            else:
                masked_token = random.choice(vocab.idx_to_token)
        mlm_input_tokens[mlm_pred_position] = masked_token
        pred_positions_and_labels.append((mlm_pred_position, tokens[mlm_pred_position]))
    return mlm_input_tokens, pred_positions_and_labels

In [22]:
# 生成遮蔽语言模型任务的数据
def _get_mlm_data_from_tokens(tokens, vocab):
    candidate_pred_positions = []
    # tokens是一个字符串列表
    for i, token in enumerate(tokens):
        # 在遮蔽语言模型任务中不会预测特殊词元
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)
    # 遮蔽语言模型任务中预测15%的随机词元
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds, vocab)
    pred_positions_and_labels = sorted(pred_positions_and_labels, key=lambda x: x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

In [23]:
# 生成BERT输入数据
def _pad_bert_inputs(examples, max_len, vocab):
    # examples包含来自两个预训练任务的辅助函数_get_nsp_data_from_paragraph和_get_mlm_data_from_tokens的输出。
    max_num_mlm_preds = round(max_len * 0.15)
    all_token_ids, all_segments, valid_lens,  = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    for token_ids, pred_positions, mlm_pred_label_ids, segments, is_next in examples:
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (max_len - len(token_ids)), dtype=torch.long))
        all_segments.append(torch.tensor(segments + [0] * (max_len - len(segments)), dtype=torch.long))
        # valid_lens不包括'<pad>'的计数
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # 填充词元的预测将通过乘以0权重在损失中过滤掉
        all_mlm_weights.append(torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (max_num_mlm_preds - len(pred_positions)), dtype=torch.float32))
        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    return all_token_ids, all_segments, valid_lens, all_pred_positions, all_mlm_weights, all_mlm_labels, nsp_labels

In [24]:
class _WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # 输入paragraphs[i]是代表段落的句子字符串列表；
        # 而输出paragraphs[i]是代表段落的句子列表，其中每个句子都是词元列表
        paragraphs = [utils.tokenize(paragraph, token='word') for paragraph in paragraphs]
        sentences = [sentence for paragraph in paragraphs for sentence in paragraph]
        self.vocab = utils.Vocab(sentences, min_freq=5, reserved_tokens=['<pad>', '<mask>', '<cls>', '<sep>'])
        # 获取下一句子预测任务的数据
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(paragraph, paragraphs, self.vocab, max_len))
        # 获取遮蔽语言模型任务的数据
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab) + (segments, is_next)) for tokens, segments, is_next in examples]
        # 填充输入
        (self.all_token_ids, self.all_segments, self.valid_lens, self.all_pred_positions, self.all_mlm_weights, 
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(examples, max_len, self.vocab)

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx], self.valid_lens[idx], self.all_pred_positions[idx],
                self.all_mlm_weights[idx], self.all_mlm_labels[idx], self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)

In [25]:
def load_data_wiki(batch_size, max_len):
    """加载WikiText-2数据集"""
    num_workers = utils.get_dataloader_workers()
    paragraphs = _read_wiki(data_dir_path)
    train_set = _WikiTextDataset(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab

In [26]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X, mlm_Y, nsp_y) in train_iter:
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape, pred_positions_X.shape, mlm_weights_X.shape, mlm_Y.shape, nsp_y.shape)
    break

torch.Size([512, 64]) torch.Size([512, 64]) torch.Size([512]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512])


In [27]:
len(vocab)

20256

### 为简单起见，句号用作拆分句子的唯一分隔符。尝试其他的句子拆分技术，比如Spacy和NLTK。以NLTK为例，需要先安装NLTK：pip install nltk。在代码中先import nltk。然后下载Punkt语句词元分析器：nltk.download('punkt')。要拆分句子，比如sentences = 'This is great ! Why not ?'，调用nltk.tokenize.sent_tokenize(sentences)将返回两个句子字符串的列表：['This is great !', 'Why not ?']。

In [28]:
# 我们加载WikiText-2数据集作为小批量的预训练样本，用于遮蔽语言模型和下一句预测。
# 批量大小是512，BERT输入序列的最大长度是64。注意，在原始BERT模型中，最大长度是512。
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

# 预训练BERT

原始BERT (Devlin et al., 2018)有两个不同模型尺寸的版本。基本模型（bert_base）使用12层（Transformer编码器块），768个隐藏单元（隐藏大小）和12个自注意头。大模型（bert_large）使用24层，1024个隐藏单元和16个自注意头。值得注意的是，前者有1.1亿个参数，后者有3.4亿个参数。
为了便于演示，我们定义了一个小的BERT，使用了2层、128个隐藏单元和2个自注意头。

In [29]:
net = BERTModel(len(vocab), num_hiddens=128, norm_shape=[128], ffn_num_input=128, ffn_num_hiddens=256, num_heads=2, num_layers=2, 
                dropout=0.2, key_size=128, query_size=128, value_size=128, hid_in_features=128, mlm_in_features=128, nsp_in_features=128)
devices = utils.try_all_gpus()
loss = nn.CrossEntropyLoss()

In [30]:
# BERT预训练的最终损失是遮蔽语言模型损失和下一句预测损失的和。
def _get_batch_loss_bert(net, loss, vocab_size, tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X, mlm_Y, nsp_y):
    # 前向传播
    _, mlm_Y_hat, nsp_Y_hat = net(tokens_X, segments_X, valid_lens_x.reshape(-1), pred_positions_X)
    # 计算遮蔽语言模型损失
    mlm_l = loss(mlm_Y_hat.reshape(-1, vocab_size), mlm_Y.reshape(-1)) * mlm_weights_X.reshape(-1, 1)
    mlm_l = mlm_l.sum() / (mlm_weights_X.sum() + 1e-8)
    # 计算下一句子预测任务的损失
    nsp_l = loss(nsp_Y_hat, nsp_y)
    l = mlm_l + nsp_l
    return mlm_l, nsp_l, l

In [31]:
# 输入num_steps指定了训练的迭代步数
def train_bert(train_iter, net, loss, vocab_size, devices, num_steps):
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])
    trainer = torch.optim.Adam(net.parameters(), lr=0.01)
    # 遮蔽语言模型损失的和，下一句预测任务损失的和，句子对的数量，计数
    mlm_l_sum = nsp_l_sum = tokens_num = count = 0
    time_used_list = list()
    num_steps_reached = False
    step = 0
    while step < num_steps and not num_steps_reached:
        for tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X, mlm_Y, nsp_y in train_iter:
            tokens_X = tokens_X.to(devices[0])
            segments_X = segments_X.to(devices[0])
            valid_lens_x = valid_lens_x.to(devices[0])
            pred_positions_X = pred_positions_X.to(devices[0])
            mlm_weights_X = mlm_weights_X.to(devices[0])
            mlm_Y, nsp_y = mlm_Y.to(devices[0]), nsp_y.to(devices[0])
            trainer.zero_grad()
            time_0 = time.time()
            mlm_l, nsp_l, l = _get_batch_loss_bert(net, loss, vocab_size, tokens_X, segments_X, valid_lens_x,
                                                   pred_positions_X, mlm_weights_X, mlm_Y, nsp_y)
            l.backward()
            trainer.step()
            mlm_l_sum += mlm_l
            nsp_l_sum += nsp_l
            tokens_num += tokens_X.shape[0]
            count += 1
            time_used_list.append(time.time() - time_0)
            if (step + 1) % 50 == 0:
                print(f'step: {step + 1}/{num_steps}, MLM loss {mlm_l_sum / count:.3f}, NSP loss {nsp_l_sum / count:.3f}')
            step += 1
            if step == num_steps:
                num_steps_reached = True
                break

    print(f'MLM loss {mlm_l_sum / count:.3f}, NSP loss {nsp_l_sum / count:.3f}')
    print(f'{tokens_num / sum(time_used_list):.1f} sentence pairs/sec on {str(devices)}')

In [32]:
train_bert(train_iter, net, loss, len(vocab), devices, 500)

step: 50/500, MLM loss 5.676, NSP loss 0.770
step: 100/500, MLM loss 5.355, NSP loss 0.733
step: 150/500, MLM loss 5.211, NSP loss 0.721
step: 200/500, MLM loss 5.132, NSP loss 0.717
step: 250/500, MLM loss 5.087, NSP loss 0.713
step: 300/500, MLM loss 5.053, NSP loss 0.711
step: 350/500, MLM loss 5.032, NSP loss 0.709
step: 400/500, MLM loss 5.023, NSP loss 0.708
step: 450/500, MLM loss 5.013, NSP loss 0.707
step: 500/500, MLM loss 5.007, NSP loss 0.706
MLM loss 5.007, NSP loss 0.706
3342.5 sentence pairs/sec on [device(type='cuda', index=0)]


# 用BERT表示文本

In [33]:
def get_bert_encoding(net, tokens_a, tokens_b=None):
    tokens, segments = get_tokens_and_segments(tokens_a, tokens_b)
    token_ids = torch.tensor(vocab[tokens], device=devices[0]).unsqueeze(0)
    segments = torch.tensor(segments, device=devices[0]).unsqueeze(0)
    valid_len = torch.tensor(len(tokens), device=devices[0]).unsqueeze(0)
    encoded_X, _, _ = net(token_ids, segments, valid_len)
    return encoded_X

In [34]:
tokens_a = ['a', 'crane', 'is', 'flying']
encoded_text = get_bert_encoding(net, tokens_a)
# 词元：'<cls>','a','crane','is','flying','<sep>'
encoded_text_cls = encoded_text[:, 0, :]
encoded_text_crane = encoded_text[:, 2, :]
encoded_text.shape, encoded_text_cls.shape, encoded_text_crane[0][:3]

(torch.Size([1, 6, 128]),
 torch.Size([1, 128]),
 tensor([-0.3926, -1.2659,  2.3238], device='cuda:0', grad_fn=<SliceBackward>))

在预训练BERT之后，我们可以用它来表示单个文本、文本对或其中的任何词元。

同一个词元在不同的上下文中具有不同的BERT表示。这支持BERT表示是上下文敏感的。

# 针对序列级和词元级应用微调BERT

## 1、单文本分类
    
在单文本分类应用中，特殊分类标记“\<cls\>”的BERT表示对整个输入文本序列的信息进行编码。作为输入单个文本的表示，它将被送入到由全连接（稠密）层组成的小多层感知机中，以输出所有离散标签值的分布。

![v54_1](../data/img/v54_1.png)

## 2、文本对分类或回归
![v54_2](../data/img/v54_2.png)

# 3、文本标注
![v54_3](../data/img/v54_3.png)

# 4、问答
![v54_4](../data/img/v54_4.png)

![v54_5](../data/img/v54_5.png)

# 自然语言推断：微调BERT

自然语言推断（natural language inference）主要研究 假设（hypothesis）是否可以从前提（premise）中推断出来， 其中两者都是文本序列。 换言之，自然语言推断决定了一对文本序列之间的逻辑关系。这类关系通常分为三种类型：

- 蕴涵（entailment）：假设可以从前提中推断出来。

- 矛盾（contradiction）：假设的否定可以从前提中推断出来。

- 中性（neutral）：所有其他情况。

自然语言推断也被称为识别文本蕴涵任务

In [35]:
def load_pretrained_model(pretrained_model, num_hiddens, ffn_num_hiddens, num_heads, num_layers, dropout, max_len, devices):
    data_dir = Path(fr'../data/{pretrained_model}.torch')
    if not data_dir.exists:
        raise Exception(f'dir:{data_dir.as_posix()} is not exists.')
    # 定义空词表以加载预定义词表
    vocab = utils.Vocab()
    vocab.idx_to_token = json.load(open((data_dir / 'vocab.json').as_posix()))
    vocab.token_to_idx = {token: idx for idx, token in enumerate(vocab.idx_to_token)}
    bert = BERTModel(len(vocab), num_hiddens, norm_shape=[256], ffn_num_input=256, ffn_num_hiddens=ffn_num_hiddens, num_heads=4, num_layers=2, dropout=0.2, 
                     max_len=max_len, key_size=256, query_size=256, value_size=256, hid_in_features=256, mlm_in_features=256, nsp_in_features=256)
    # 加载预训练BERT参数
    bert.load_state_dict(torch.load((data_dir / 'pretrained.params').as_posix()))
    return bert, vocab

In [36]:
devices = utils.try_all_gpus()
bert, vocab = load_pretrained_model('bert.small', num_hiddens=256, ffn_num_hiddens=512, num_heads=4, 
                                    num_layers=2, dropout=0.1, max_len=512, devices=devices)

### 微调BERT的数据集

In [37]:
# 斯坦福自然语言推断（SNLI）数据集
snli_data_dir = Path(r'../data/snli_1.0')
# 列名
# gold_label, sentence1_binary_parse, sentence2_binary_parse, sentence1_parse, sentence2_parse, sentence1, sentence2, captionID, pairID, label1, label2, label3, label4, label5

def read_snli(data_dir: Path, is_train):
    """将SNLI数据集解析为前提、假设和标签"""
    def extract_text(s):
        # 删除我们不会使用的信息
        s = re.sub('\\(', '', s)
        s = re.sub('\\)', '', s)
        # 用一个空格替换两个或多个连续的空格
        s = re.sub('\\s{2,}', ' ', s)
        return s.strip()
    label_set = {'entailment': 0, 'contradiction': 1, 'neutral': 2}
    file_name = data_dir / ('snli_1.0_train.txt' if is_train else 'snli_1.0_test.txt')
    with open(file_name.as_posix(), 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]
    premises = [extract_text(row[1]) for row in rows if row[0] in label_set]
    hypotheses = [extract_text(row[2]) for row in rows if row[0] in label_set]
    labels = [label_set[row[0]] for row in rows if row[0] in label_set]
    return premises, hypotheses, labels

In [38]:
train_data = read_snli(snli_data_dir, is_train=True)
for x0, x1, y in zip(train_data[0][:3], train_data[1][:3], train_data[2][:3]):
    print('前提：', x0)
    print('假设：', x1)
    print('标签：', y)

前提： A person on a horse jumps over a broken down airplane .
假设： A person is training his horse for a competition .
标签： 2
前提： A person on a horse jumps over a broken down airplane .
假设： A person is at a diner , ordering an omelette .
标签： 1
前提： A person on a horse jumps over a broken down airplane .
假设： A person is outdoors , on a horse .
标签： 0


In [39]:
# 训练集和测试集中的三个标签“蕴涵”“矛盾”和“中性”数量是平衡的。
test_data = read_snli(snli_data_dir, is_train=False)
for data in [train_data, test_data]:
    print([[row for row in data[2]].count(i) for i in range(3)])

[183416, 183187, 182764]
[3368, 3237, 3219]


In [40]:
class SNLIDataset(torch.utils.data.Dataset):
    """用于加载SNLI数据集的自定义数据集"""
    def __init__(self, dataset, num_steps, vocab=None):
        self.num_steps = num_steps
        all_premise_tokens = utils.tokenize(dataset[0], token='word')
        all_hypothesis_tokens = utils.tokenize(dataset[1], token='word')
        if vocab is None:
            self.vocab = utils.Vocab(all_premise_tokens + all_hypothesis_tokens, min_freq=5, reserved_tokens=['<pad>'])
        else:
            self.vocab = vocab
        self.premises = self._pad(all_premise_tokens)
        self.hypotheses = self._pad(all_hypothesis_tokens)
        self.labels = torch.tensor(dataset[2])
        print('read ' + str(len(self.premises)) + ' examples')

    def _pad(self, lines):
        return torch.tensor([utils.truncate_pad(self.vocab[line], self.num_steps, self.vocab['<pad>']) for line in lines])

    def __getitem__(self, idx):
        return (self.premises[idx], self.hypotheses[idx]), self.labels[idx]

    def __len__(self):
        return len(self.premises)

In [41]:
def load_data_snli(batch_size, num_steps=50):
    """加载SNLI数据集并返回数据迭代器和词表"""
    num_workers = utils.get_dataloader_workers()
    train_data = read_snli(snli_data_dir, True)
    test_data = read_snli(snli_data_dir, False)
    train_set = SNLIDataset(train_data, num_steps)
    test_set = SNLIDataset(test_data, num_steps, train_set.vocab)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True, num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(test_set, batch_size, shuffle=False, num_workers=num_workers)
    return train_iter, test_iter, train_set.vocab

In [45]:
train_iter, test_iter, vocab = load_data_snli(batch_size=128, num_steps=50)
len(vocab)

read 549367 examples
read 9824 examples


18678

In [46]:
for X, Y in train_iter:
    print(X[0].shape)
    print(X[1].shape)
    print(Y.shape)
    break

torch.Size([128, 50])
torch.Size([128, 50])
torch.Size([128])


In [47]:
train_iter = None
test_iter = None

In [48]:
# 做成 bert 数据集
class SNLIBERTDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, max_len, vocab=None):
        all_premise_hypothesis_tokens = [
            [p_tokens, h_tokens] for p_tokens, h_tokens in zip(
                *[utils.tokenize([s.lower() for s in sentences], token='word') for sentences in dataset[:2]]
            )
        ]
        print('- SNLIBERTDataset dataset loaded -')
        self.labels = torch.tensor(dataset[2])
        self.vocab = vocab
        self.max_len = max_len
        self.all_token_ids, self.all_segments, self.valid_lens = self._preprocess(all_premise_hypothesis_tokens)
        print('preprocess over, read ' + str(len(self.all_token_ids)) + ' examples')

    def _preprocess(self, all_premise_hypothesis_tokens):
        # pool = multiprocessing.Pool(4)  # 使用多进程
        # out = pool.map(self._mp_worker, all_premise_hypothesis_tokens)
        out = map(self._mp_worker, all_premise_hypothesis_tokens)
        all_token_ids = list()
        all_segments = list()
        valid_lens = list()
        for token_ids, segments, valid_len in out:
            all_token_ids.append(token_ids)
            all_segments.append(segments)
            valid_lens.append(valid_len)
        return torch.tensor(all_token_ids, dtype=torch.long), torch.tensor(all_segments, dtype=torch.long), torch.tensor(valid_lens)

    def _mp_worker(self, premise_hypothesis_tokens):
        p_tokens, h_tokens = premise_hypothesis_tokens
        p_tokens, h_tokens = self._truncate_pair_of_tokens(p_tokens, h_tokens)
        tokens, segments = get_tokens_and_segments(p_tokens, h_tokens)
        token_ids = self.vocab[tokens] + [self.vocab['<pad>']] * (self.max_len - len(tokens))
        segments = segments + [0] * (self.max_len - len(segments))
        valid_len = len(tokens)
        return token_ids, segments, valid_len

    def _truncate_pair_of_tokens(self, p_tokens, h_tokens):
        # 为BERT输入中的'<CLS>'、'<SEP>'和'<SEP>'词元保留位置
        p_h_tokens_len = len(p_tokens) + len(h_tokens)
        if p_h_tokens_len > self.max_len - 3:
            # 需弹出的数量
            pop_num = p_h_tokens_len - (self.max_len - 3)
            # 每个应从尾部去掉的数量
            half_pop_num = pop_num//2
            if pop_num % 2 == 0:
                p_tokens = p_tokens[:-half_pop_num]
                h_tokens = h_tokens[:-half_pop_num]
            else:
                p_tokens = p_tokens[:-half_pop_num]
                h_tokens = h_tokens[:-(half_pop_num+1)]
        return p_tokens, h_tokens

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx], self.valid_lens[idx]), self.labels[idx]

    def __len__(self):
        return len(self.all_token_ids)

In [49]:
# 如果出现显存不足错误，请减少“batch_size”。在原始的BERT模型中，max_len=512
batch_size, max_len, num_workers = 512, 128, utils.get_dataloader_workers()
train_set = SNLIBERTDataset(read_snli(snli_data_dir, True), max_len, vocab)
test_set = SNLIBERTDataset(read_snli(snli_data_dir, False), max_len, vocab)
train_iter = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(test_set, batch_size, num_workers=num_workers)

- SNLIBERTDataset dataset loaded -
preprocess over, read 549367 examples
- SNLIBERTDataset dataset loaded -
preprocess over, read 9824 examples


# 微调BERT

用于自然语言推断的微调BERT只需要一个额外的多层感知机，该多层感知机由两个全连接层组成（请参见下面BERTClassifier类中的self.hidden和self.output）。这个多层感知机将特殊的“\<cls\>”词元的BERT表示进行了转换，该词元同时编码前提和假设的信息为自然语言推断的三个输出：蕴涵、矛盾和中性。

In [50]:
class BERTClassifier(nn.Module):
    def __init__(self, bert):
        super(BERTClassifier, self).__init__()
        self.encoder = bert.encoder
        self.hidden = bert.hidden
        self.output = nn.Linear(256, 3)

    def forward(self, inputs):
        tokens_X, segments_X, valid_lens_x = inputs
        encoded_X = self.encoder(tokens_X, segments_X, valid_lens_x)
        return self.output(self.hidden(encoded_X[:, 0, :]))

在BERT微调的常见实现中，只有额外的多层感知机（net.output）的输出层的参数将从零开始学习。预训练BERT编码器（net.encoder）和额外的多层感知机的隐藏层（net.hidden）的所有参数都将进行微调。

In [51]:
net = BERTClassifier(bert)

In [52]:
lr, num_epochs = 1e-4, 5
trainer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss(reduction='none')
utils.train_gpus(net, train_iter, test_iter, loss, trainer, num_epochs, devices)

training on: [device(type='cuda', index=0)], [2024-09-14 16:41:57]
epoch: 1/5, loss: 140.799, train_acc: 0.624, test_acc: 0.705, epoch_time: [0:00:39:53]
epoch: 2/5, loss: 118.024, train_acc: 0.708, test_acc: 0.737, epoch_time: [0:00:39:32]
epoch: 3/5, loss: 107.830, train_acc: 0.737, test_acc: 0.751, epoch_time: [0:00:41:28]
epoch: 4/5, loss: 100.259, train_acc: 0.758, test_acc: 0.758, epoch_time: [0:00:41:55]
epoch: 5/5, loss: 94.561, train_acc: 0.774, test_acc: 0.766, epoch_time: [0:00:41:56]
*** training speed: 1.3 examples/sec on [device(type='cuda', index=0)], all_time: [0:03:24:47] ***


In [53]:
def bert_predict_snli(net, vocab, premise_bat, hypothesis_bat, max_len, devices):
    """
        推断前提和假设之间的逻辑关系
    """
    net.eval()
    token_ids = list()
    segments = list()
    valid_len = list()
    for p_tokens, h_tokens in zip(premise_bat, hypothesis_bat):
        # 截断到定长内，为BERT输入中的'<CLS>'、'<SEP>'和'<SEP>'词元保留位置
        while len(p_tokens) + len(h_tokens) > max_len - 3:
            if len(p_tokens) > len(h_tokens):
                p_tokens.pop()
            else:
                h_tokens.pop()
        # 拼接<cls>/<sep>形成bert的tokens和segments
        _tokens, _segments = get_tokens_and_segments(p_tokens, h_tokens)
        # 填充<pad>
        _token_ids = vocab[_tokens] + [vocab['<pad>']] * (max_len - len(_tokens))
        _segments = _segments + [0] * (max_len - len(_segments))
        _valid_len = len(_tokens)
       
        # print(_token_ids)
        # print(_segments)
        # print(_valid_len)
        
        token_ids.append(torch.tensor(_token_ids))
        segments.append(torch.tensor(_segments))
        valid_len.append(torch.tensor(_valid_len))
    
    token_ids = torch.stack(token_ids, dim=0).to(devices[0])
    segments = torch.stack(segments, dim=0).to(devices[0])
    valid_len = torch.stack(valid_len, dim=0).to(devices[0])
    
    predict_label = torch.argmax(net(inputs=(token_ids, segments, valid_len)), dim=1)
    print(predict_label)

In [54]:
test_sentence_list = [
    ['I love you very much .', 'I have feelings for you .'],
    ['he likes to jump .', 'he likes sports .'],
    ['This burger is delicious .', 'The sky is blue .'],
    ['I am very hungry today .', 'I can not eat anything today .'],
    ['I am full of energy .', 'I am tired .']
]

In [55]:
p_sentence_list = list()
h_sentence_list = list()
for pair_sentence_list in test_sentence_list:
    p_sentence_list.append(pair_sentence_list[0].split())
    h_sentence_list.append(pair_sentence_list[1].split())
p_sentence_list, h_sentence_list

([['I', 'love', 'you', 'very', 'much', '.'],
  ['he', 'likes', 'to', 'jump', '.'],
  ['This', 'burger', 'is', 'delicious', '.'],
  ['I', 'am', 'very', 'hungry', 'today', '.'],
  ['I', 'am', 'full', 'of', 'energy', '.']],
 [['I', 'have', 'feelings', 'for', 'you', '.'],
  ['he', 'likes', 'sports', '.'],
  ['The', 'sky', 'is', 'blue', '.'],
  ['I', 'can', 'not', 'eat', 'anything', 'today', '.'],
  ['I', 'am', 'tired', '.']])

In [56]:
predict_label = bert_predict_snli(net, vocab, p_sentence_list, h_sentence_list, max_len, devices)

tensor([0, 0, 1, 1, 0], device='cuda:0')
